In [4]:
%cd ~/Workspace/VisualText/

/home/chengjiun/Workspace/VisualText


In [5]:
import zipfile
import requests, zipfile, io
import os

zip_file_url = "https://datascience-models-ramsri.s3.amazonaws.com/t5_paraphraser.zip"
folder_path = zip_file_url.split("/")[-1].replace(".zip","")

if not os.path.exists(folder_path):
    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall('./t5_paraphraser')
else:
    print ("Folder available: ",folder_path)

In [6]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:

model = T5ForConditionalGeneration.from_pretrained('./t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

In [14]:
def paraphrasing(sentence, tokenizer=tokenizer, model=model, text=None):
    if text is None:
        text =  "paraphrase: " + sentence + " </s>"

    max_len = 256

    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
    beam_outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        do_sample=True,
        max_length=256,
        top_k=120,
        top_p=0.98,
        early_stopping=True,
        num_return_sequences=10
    )


    print ("\nOriginal Question ::")
    print (sentence)
    print ("\n")
    print ("Paraphrased Questions :: ")
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(sent)

    for i, final_output in enumerate(final_outputs):
        print("{}: {}".format(i, final_output))
    return final_output

In [10]:



sentence = "Which course should I take to get started in data science?"
# sentence = "What are the ingredients required to bake a perfect cake?"
# sentence = "What is the best possible approach to learn aeronautical engineering?"
# sentence = "Do apples taste better than oranges in general?"
output = paraphrasing(sentence)


Original Question ::
Which course should I take to get started in data science?


Paraphrased Questions :: 
0: What course should I take to start data science?
1: Which course should I take in data science?
2: What is the best online / offline course to study data science?
3: What course should I take to get started with the data science?
4: Which is the best course for data science?
5: What is best course for data science?
6: What course should I take in Data Science to get started?
7: Which are the best courses for the data science field?
8: Which course should I take to be good in data science?
9: What is the best way to learn data science?


In [11]:
sentence = "Avoid displaying weapons in pictures."

output = paraphrasing(sentence)


Original Question ::
Avoid displaying weapons in pictures.


Paraphrased Questions :: 
0: Should you display your weapons in a picture?
1: Avoid displaying weapon in a picture.
2: The world needs new weapons, so avoid displaying them in real life.
3: Avoid using weapons in pictures.
4: This site is aimed at military users, do not include weapons in pictures.
5: Avoid showing weapons or weapons with pictures.
6: Have a knife in your picture?
7: Is it OK to show weapons in public as pictures?
8: Avoid displaying weapons on pictures.


In [12]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)


device  cuda


In [16]:
sentence = "Avoid displaying weapons in pictures."

output = paraphrasing(sentence, model=model, tokenizer=tokenizer, text='QQsentence)


Original Question ::
Avoid displaying weapons in pictures.


Paraphrased Questions :: 
0: Avoid showing Avoid displaying at displaying weapons in pictures. Al displaying displaying displaying weapons De Avoid displaying other displaying weapons in photographs. Blu displaying weapons in Pictures Pictures Without Avoid displaying the displaying displaying weapons in Pictures in Pictures: Avoid displaying avoid displaying displaying displaying weapons or Pictures Ver displaying displaying displaying weapons displaying weapons in Pictures. Avoid displaying weapons in Picture Picture Avoid displaying Arm displaying photographs displaying Alcohol displaying weapons in Pictures - Avoid displaying displaying weapons the displaying
1: Avoid displaying displaying on displaying weapons in picture. Avoid Avoid displaying weapons from Picture Picture Don displaying weapons Avoid displaying: Avoid Vi Vi Display displaying displaying displaying graphic weapons in pictures. Avoid displaying displayin